# Fine Tuning of ResNet50

We want to fine-tune ResNet50 to classify pictorial genres.

We then want to save the model.

## Libraries

In [ ]:
!pip install tensorflow-addons

In [ ]:
import tensorflow_addons as tfa

In [ ]:
from tensorflow.keras.models import Model
from keras.applications.resnet import ResNet50 
from keras.applications.resnet import preprocess_input as preprocess_input_resnet
from tensorflow.keras.preprocessing import image as image_resnet


import tensorflow as tf
from keras.layers import *
import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import numpy as np
import pandas as pd
import cv2 as cv

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import os

import itertools

from IPython.display import clear_output

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = '/content/drive/MyDrive'

In [ ]:
base_dir = root_dir + '/Painting/data/'
data_folder = base_dir + 'raw/dataset/'
train_folder = data_folder + 'train/'
test_folder = data_folder + 'test/'

## Dataset

We have to resize to (224,224) and to preprocess all the images to be readable from ResNet50.

In [ ]:
def preprocess_cv2_image_resnet(image):
  #image = cv.imread(image_path)
  image = cv.resize(image, (224, 224))
  image =  cv.cvtColor(image, cv.COLOR_BGR2RGB)
  image = Image.fromarray(image)
  image = image_resnet.img_to_array(image)
  image = np.expand_dims(image, axis = 0)
  return preprocess_input_resnet(image)

In [ ]:
def preprocess_resnet(image):
  image = cv.resize(image, (224, 224))
  image =  cv.cvtColor(image, cv.COLOR_BGR2RGB)
  image = Image.fromarray( (image * 255).astype(np.uint8) )
  image = image_resnet.img_to_array(image)
  #image = np.expand_dims(image, axis = 0)
  image =  preprocess_input_resnet(image)

  image = image * 255
  image = image.astype(np.uint8)
  return image

In [ ]:
image_path = train_folder + '29998.jpg'
image = cv.imread(image_path)
print( image.shape )
image_p = preprocess_cv2_image_resnet(image)
print( image_p.shape )
image_p = preprocess_resnet(image)
print( image_p.shape )

In [ ]:
def dataframe_generator():
  df = pd.read_csv(data_folder + "all_data_info.csv")
  df.rename(columns={"new_filename": "filename"}, inplace=True)
  df.drop(columns=["pixelsx", "pixelsy", "size_bytes", "artist_group", "source"], inplace=True)
  df.drop(columns=["artist", "style", "date", "title"], inplace=True)
  df.dropna(subset=["genre"], inplace=True)
  df.reset_index(drop=True, inplace=True)

  # save memory 
  df["genre"] = df["genre"].astype("category")

  print(df.memory_usage(deep=True))
  print(df.info())

  df.reset_index(drop=True, inplace=True)
  df.to_csv(data_folder + "not_all_data_info.csv")
  return df

In [ ]:
#df = dataframe_generator() # To generate it
df = pd.read_csv(data_folder + "not_all_data_info.csv") # We have it already

In [ ]:
pd.set_option('display.expand_frame_repr', False)
print(df.columns)
print(df.head())

In [ ]:
def get_image_index(filename):
  for index in range(df.shape[0]): #n_row
    if( df["filename"][index] == filename ):
      return index
  return -1

In [ ]:
def get_genre_by_filename(filename):
  index = get_image_index(filename)
  return df["genre"][index] 

In [ ]:
train_dir = os.path.join(data_folder, "resized_train")
test_dir = os.path.join(data_folder, "resized_test")

print(train_dir)
print(test_dir)

In [ ]:
def separate_image_in_folders(folder, save_folder_name):
  save_folder = os.path.abspath(os.path.join(folder, os.pardir))
  save_folder = os.path.join(save_folder, save_folder_name)

  if not os.path.exists( save_folder ):
    os.makedirs( save_folder )

  N = len( os.listdir(folder) )
  n_file = 0

  for filename in os.listdir(folder):
    img = cv.imread(os.path.join(folder,filename))
    try:
      img = cv.resize(img, (224, 224))
    except Exception as e:
      print(str(e))
      print("Problem with image: " +filename)
      img = None


    if n_file % 10 == 0:
      clear_output(wait=True)
      print("{} / {} " .format(n_file, N))
    n_file = n_file + 1 

    if img is not None:
      index = get_image_index(filename)
      if index != -1:
        genre = df["genre"][index]
        temp_path = os.path.join(save_folder, genre)
        if not os.path.exists( temp_path ):
          os.makedirs( temp_path )
        cv.imwrite( os.path.join(temp_path, filename) , img)
  
  clear_output(wait=True)
  print("{} / {} " .format(N, N))
  return True

In [ ]:
#separate_image_in_folders( train_folder, "refactored_train") # 1:30h circa

In [ ]:
separate_image_in_folders( test_folder, "refactored_test") # 5h circa

In [ ]:
train_dir = os.path.join(data_folder, "refactored_train")
test_dir = os.path.join(data_folder, "refactored_test")

In [ ]:
train_datagen = ImageDataGenerator(validation_split=0.1, 
                                   preprocessing_function=preprocess_resnet)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_resnet)

In [ ]:
batch_size = 32

num_classes = 42 #n_genre
input_shape = (224, 224) #(224, 224, 3)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=input_shape,
        color_mode='rgb',
        batch_size=batch_size,
        subset='training',
        class_mode='sparse',
        )

validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=input_shape,
        color_mode='rgb',
        batch_size=batch_size,
        subset='validation',
        class_mode='sparse',
        )
"""
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=input_shape,
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='categorical'
        )
"""

## Models

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)) )
#base_model.summary()

In [ ]:
head_model = base_model.output
head_model = AveragePooling2D(pool_size=(7, 7))(head_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(256, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense( num_classes , activation="softmax")(head_model)

In [ ]:
model = Model(inputs=base_model.input, outputs=head_model)
#model.summary()

In [ ]:
for layer in base_model.layers:
	layer.trainable = False

In [ ]:
f1_micro = tfa.metrics.F1Score(num_classes=num_classes, average='micro') # Unbalanced
f1_macro = tfa.metrics.F1Score(num_classes=num_classes, average='macro')

In [ ]:
# compile the model
model.compile(loss="sparse_categorical_crossentropy", #sparse_categorical_crossentropy # categorical_crossentropy
              optimizer="adam",
              metrics=["accuracy", f1_micro] )

# Train

In [ ]:
# Train it on the data for some epochs
epochs = 10

history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

In [ ]:
model.save(data_folder + 'resnet_finetuing_model')

In [ ]:
from matplotlib import pyplot as plt
#x_plot = list(range(1,epochs+1))

def plot_history(network_history):
    epochs = len( history.history['loss'] )
    x_plot = list(range(1,epochs+1))
    
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(x_plot, network_history.history['loss'])
    plt.plot(x_plot, network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(x_plot, network_history.history['accuracy'])
    plt.plot(x_plot, network_history.history['val_accuracy'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
eval_result = model.evaluate(test_generator)
print("[test loss, test accuracy]:", eval_result)

In [ ]:
test_pred = model.predict(test_generator)

In [ ]:
y_pred = []
for pred in test_pred:
  y_pred.append( np.argmax(pred) )

In [ ]:
from sklearn.metrics import f1_score

y_true = test_generator.labels

f1_test = f1_score(y_true, y_pred, average='macro')
print('Average f1_score: {} \n' .format(f1_test) )

print('F1-SCORE FOR EACH CLASS')
print('-----------------------')
av_f1_score = f1_score(y_true, y_pred, average=None)
for i in range(len(av_f1_score)):
  print('{} : {} '.format( i, av_f1_score[i]))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay( confusion_matrix=cm )

disp.plot()
frame1 = plt.gca()
frame1.axes.get_xaxis().set_visible(False)
plt.show()